In [1]:
import sys
import configparser
config = configparser.ConfigParser()
config.read_file(open("../../settings.ini"))

sys.path.insert(0, config.get('PATHS','libs_path'))
engine_path = config.get('DATABASE','engine_path')

In [2]:
local_path = "../data_final/"
remote_path = "/"

In [3]:
# from local file postgres.py
import postgres
# from local file commons.py
from commons import inline_table_xml, inline_dimension_xml, download_file, download_zip_file, extract_zip_file

import json
import pandas as pd
from sqlalchemy import create_engine

In [4]:
df1 = download_file(remote_path, local_path, "population_census.csv")

Already downloaded. Using: ../data_final/population_census.csv


In [5]:
df1.head()

,area_id,district_id,scholarship,sex,age,economy_activity,birth_place,children_currently_alive,residence_country_5_years_ago,birth_comuna,...,residence_comuna_5_years_ago,residence_comuna,year_of_arrival_to_chile,habitual_residence,formal_education,birth_country,cant_per,comuna_datachile_id,comuna_customs_id,aboriginal_people
0,1,1,0,2,0,98,1,98,998,98,...,98,98,9998,1,2,998,1,113,1101.0,1
1,1,1,0,2,0,98,1,98,998,98,...,98,98,9998,1,3,998,1,113,1101.0,2
2,1,1,0,2,0,98,1,98,998,98,...,98,98,9998,1,3,998,1,113,1101.0,7
3,1,1,0,2,0,98,1,98,998,98,...,98,98,9998,1,1,998,6,113,1101.0,98
4,1,1,0,2,0,98,1,98,998,98,...,98,98,9998,1,3,998,13,113,1101.0,98


In [6]:
df1 = df1.rename(columns = {"comuna_datachile_id": "comuna_id"})

In [7]:
engine = create_engine(engine_path)
db = postgres.PostgresDriver(engine)
db.to_sql(df1, "census", "fact_population_census")

DROP TABLE IF EXISTS census.fact_population_census;
CREATE TABLE "census"."fact_population_census" (
"area_id" INTEGER,
  "district_id" INTEGER,
  "scholarship" INTEGER,
  "sex" INTEGER,
  "age" INTEGER,
  "economy_activity" TEXT,
  "birth_place" INTEGER,
  "children_currently_alive" INTEGER,
  "residence_country_5_years_ago" INTEGER,
  "birth_comuna" INTEGER,
  "residence_country" INTEGER,
  "native_people" INTEGER,
  "highest_level_approved" INTEGER,
  "residence_5_years_ago" INTEGER,
  "highest_course_approved" INTEGER,
  "children_born_alive" INTEGER,
  "residence_comuna_5_years_ago" INTEGER,
  "residence_comuna" INTEGER,
  "year_of_arrival_to_chile" INTEGER,
  "habitual_residence" INTEGER,
  "formal_education" INTEGER,
  "birth_country" INTEGER,
  "cant_per" INTEGER,
  "comuna_id" INTEGER,
  "comuna_customs_id" REAL,
  "aboriginal_people" INTEGER
)
COPY "census"."fact_population_census" ("area_id","district_id","scholarship","sex","age","economy_activity","birth_place","children_c

In [8]:
engine.execute("""
CREATE INDEX fact_population_census_index 
ON census.fact_population_census (comuna_id)
""")

In [9]:
# Read questionnaire file
questionnaire = pd.ExcelFile("https://docs.google.com/spreadsheets/d/e/2PACX-1vQ4xZxDpyDY4NursNbrsWlCqjREdmBbfC1EMlz4UGQe7M8wMA7Mqw8tZbAcBdkjgBzloyQdcnhiv10C/pub?output=xlsx")
labels = pd.read_excel(questionnaire, "Labels")

for label in list(df1):
    q = labels[labels["label"] == label].iloc[0]["id"]
    if q[0] == "_" or q[0] == "P":
        df = pd.read_excel(questionnaire, q)
        print (inline_dimension_xml(df, label, "id", "es", label))


<Dimension name="Sex" foreignKey="sex">
  <Hierarchy hasAll="true">
            
<InlineTable alias="sex">
  <ColumnDefs>
    <ColumnDef name="id" type="Numeric" />
    <ColumnDef name="description" type="String" />
    <ColumnDef name="es_description" type="String" />
  </ColumnDefs>
  <Rows>
    <Row>
      <Value column="id">1</Value>
      <Value column="description">Mujer</Value>
      <Value column="es_description">Mujer</Value>
    </Row>
  <Row>
      <Value column="id">2</Value>
      <Value column="description">Hombre</Value>
      <Value column="es_description">Hombre</Value>
    </Row>
  </Rows>
</InlineTable>
    
    <Level name="Sex" column="id" nameColumn="description" uniqueMembers="true">
      <Annotations>
        <Annotation name="es_caption">Description ES</Annotation>
      </Annotations>
      <Property name="Description ES" column="es_description" />
    </Level>
  </Hierarchy>
</Dimension>
    

<Dimension name="Age" foreignKey="age">
  <Hierarchy hasAll="tru

IndexError: single positional indexer is out-of-bounds